## ROOT Implementation

In [1]:
import ROOT
import json 

RDataFrame = ROOT.RDF.Experimental.Distributed.OSCAR.RDataFrame
RunGraphs = ROOT.RDF.Experimental.Distributed.RunGraphs

oscarclient = {
    "minio_endpoint": "minio.busy-jackson2.im.grycap.net",
    "minio_access": "minio",
    "minio_secret": "25BbLEYKlVP97g"
}

Welcome to JupyROOT 6.27/01


In [2]:
def dimuon_analysis(df):
    # For simplicity, select only events with exactly two muons and require opposite charge
    df_2mu = df.Filter("nMuon == 2", "Events with exactly two muons")
    df_os = df_2mu.Filter("Muon_charge[0] != Muon_charge[1]", "Muons with opposite charge")
     
    # Compute invariant mass of the dimuon system
    df_mass = df_os.Define("Dimuon_mass", "InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")
     
    # Make histogram of dimuon mass spectrum. Note how we can set titles and axis labels in one go.
    h = df_mass.Histo1D(("Dimuon_mass", "Dimuon mass;m_{#mu#mu} (GeV);N_{Events}", 30000, 0.25, 300), "Dimuon_mass")
     
    # Produce plot
    ROOT.gStyle.SetOptStat(0); ROOT.gStyle.SetTextFont(42)
    c = ROOT.TCanvas("c", "", 800, 700)
    c.SetLogx(); c.SetLogy()
    
    watch = ROOT.TStopwatch()
    h.SetTitle("")
    print(f"Time elapsed {watch.RealTime()}")
    h.GetXaxis().SetTitleSize(0.04)
    h.GetYaxis().SetTitleSize(0.04)
    h.Draw()
     
    label = ROOT.TLatex(); label.SetNDC(True)
    label.DrawLatex(0.175, 0.740, "#eta")
    label.DrawLatex(0.205, 0.775, "#rho,#omega")
    label.DrawLatex(0.270, 0.740, "#phi")
    label.DrawLatex(0.400, 0.800, "J/#psi")
    label.DrawLatex(0.415, 0.670, "#psi'")
    label.DrawLatex(0.485, 0.700, "Y(1,2,3S)")
    label.DrawLatex(0.755, 0.680, "Z")
    label.SetTextSize(0.040); label.DrawLatex(0.100, 0.920, "#bf{CMS Open Data}")
    label.SetTextSize(0.030); label.DrawLatex(0.630, 0.920, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}")
     
    c.SaveAs("dimuon_spectrum.pdf")

In [3]:
# Create dataframe from NanoAOD files
filenames = ["root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"]
treename = "Events"
df = RDataFrame(treename, filenames, oscarclient=oscarclient, npartitions=2)
dimuon_analysis(df)

[1, 0]
Waiting for final result, sleeping 1 second.
Waiting for final result, sleeping 1 second.


KeyboardInterrupt: 